## KNN Classifier implementation
Classifier implementing the k-nearest neighbors vote. 
Brute-force search is used to compute the nearest neighbors.

#### Input
I use .npy file containing data in the form of a dictionary:
{'X_train': array([[],[]]), 'X_test': array([[],[]]), 'y_train': array([])}. 

Target values (class labels) are exepected to be consecutive integer values starting from 0.

Can easily be altered for any input format.

In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [3]:
class KnnBruteClassifier(object):
    '''Classifier implementing the k-nearest neighbors vote. 
    Brute-force search is used to compute the nearest neighbors.
    
    Parameters
    ----------
    n_neighbors : int, optional
        Number of nearest neighbors used in voting.
    weights : str, optional (default = 'uniform')
        Weights used in voting.
        - 'uniform' :  assigns uniform weights to each neighbor
        - 'distance' : assigns weights proportional to the inverse of the distance from the query point
    metric: Metric to use for distance computation (default = 'l2').
    '''
    def __init__(self, n_neighbors=1, weights='uniform', metric="l2"):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric

     
    def fit(self, x, y):
        self.x_train = np.copy(x)
        self.y_train = np.copy(y)
        self.n_classes = self.y_train.max() + 1

        return self
        
    def predict(self, X):
        res = self.predict_proba(X)

        return res.argmax(axis=1)

        
    def predict_proba(self, X):
        if self.weights == "uniform":
            neigh_dist = None
            neigh_indarray = self.kneighbors(X, self.n_neighbors)[1]  

        if self.weights == 'distance':
            neigh_dist, neigh_indarray = self.kneighbors(X, self.n_neighbors)
            
        weights_arr = self.get_weights(X, neigh_dist, neigh_indarray) 
        normalized_weights_arr = weights_arr / np.sum(weights_arr, axis=1)[:, np.newaxis]
        
        proba = []
        for i, row in enumerate(normalized_weights_arr):
            row_pred = self.y_train[neigh_indarray[i]]
            for k in range(self.n_classes):                               
                indices = np.where(row_pred == k)
                prob_ind = np.sum(row[indices])
                proba.append(np.array(prob_ind))
        predict_proba = np.array(proba).reshape(X.shape[0], self.n_classes)

        return predict_proba       

           
    def kneighbors(self, x, n_neighbors):
        neight_dist = []
        neigh_indarray = []
        point_dist = [self.l2_norm(x_test, self.x_train) for x_test in x]

        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[:self.n_neighbors]        

            ind_list = [tup[0] for tup in sorted_neigh]                                         
            dist_list = [tup[1] for tup in sorted_neigh]

            neight_dist.append(dist_list)
            neigh_indarray.append(ind_list)
        
        return np.array(neight_dist), np.array(neigh_indarray)


    def l2_norm (self, a, b):
        return np.sqrt(np.sum((a - b)**2, axis=1))


    def get_weights(self, X, neigh_dist, neigh_indarray):
        if self.weights == "uniform":
            weights_arr = np.ones_like(neigh_indarray)

        if self.weights == 'distance':
            weights_arr = 1 / neigh_dist                    
            if np.any(np.isinf(weights_arr)):
                weights_arr = np.isinf(weights_arr).astype(int)     
     
        return weights_arr

      

In [24]:
def load_file(filename):
    data = np.load(filename, allow_pickle=True)
    return data.item()

In [25]:
input_filename = "knn_data_023.npy"
data_dict = load_file(input_filename)

In [6]:
data_dict['X_test'].shape

(330, 4)

In [7]:
data_dict['X_train'].shape

(670, 4)

First test

In [8]:
model = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_n5_y_predict = model.predict(data_dict["X_test"])

In [9]:
og_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='brute')
og_model.fit(data_dict["X_train"], data_dict["y_train"])
og_l2_uniform_n5_y_predict = og_model.predict(data_dict["X_test"])

In [12]:
all(l2_uniform_n5_y_predict == og_l2_uniform_n5_y_predict)

True

Second test

In [13]:
model = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_10_y_predict = model.predict(data_dict["X_test"])

In [14]:
og_model = KNeighborsClassifier(n_neighbors=10, weights='uniform', algorithm='brute')
og_model.fit(data_dict["X_train"], data_dict["y_train"])
og_l2_uniform_10_y_predict = og_model.predict(data_dict["X_test"])

In [15]:
all(l2_uniform_10_y_predict == og_l2_uniform_10_y_predict)

True

Third test

In [16]:
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_distance_n5_y_predict = model.predict(data_dict["X_test"])

In [17]:
og_model = KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute')
og_model.fit(data_dict["X_train"], data_dict["y_train"])
og_l2_distance_n5_y_predict = og_model.predict(data_dict["X_test"])

In [18]:
all(l2_distance_n5_y_predict == og_l2_distance_n5_y_predict)

True

In [ ]:
# output_filename = "results.npy"
# result_dict = {
#     "input_filename": input_filename,
#     "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
#     "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
#     "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
# }
# np.save(output_filename, result_dict)